In [1]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
import random
import pandas as pd

In [2]:
df = pd.read_csv('/Users/pauliuskomskis/Desktop/github/greenzone/data/2019/new_jersey-clean.csv')
pd.set_option('display.max_rows', 1000)

In [3]:
df.info

<bound method DataFrame.info of      Unnamed: 0                                     city  population  \
0             0                        Aberdeen Township       18700   
1             1                                  Absecon        8436   
2             2                                Allendale        6795   
3             3                               Allenhurst         484   
4             4                                Allentown        1784   
5             5                                    Alpha        2262   
6             6                                   Alpine        1856   
7             7                         Andover Township        5870   
8             8                              Asbury Park       15437   
9             9                            Atlantic City       37593   
10           10                       Atlantic Highlands        4308   
11           11                                  Audubon        8601   
12           12                 

In [4]:
df.head()

,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson
0,0,Aberdeen Township,18700,15,0,3,5,7,164,7,145,12,0
1,1,Absecon,8436,8,0,1,2,5,236,24,207,5,0
2,2,Allendale,6795,0,0,0,0,0,38,8,26,4,0
3,3,Allenhurst,484,0,0,0,0,0,11,1,9,1,0
4,4,Allentown,1784,0,0,0,0,0,8,2,6,0,0


In [5]:
import re

In [11]:


df['city'] = df['city'].str.replace('\d+', '').str.strip()
df['city'] = df['city'].str.replace(",", "")
df['city'] = df['city'].str.replace("-", "")
city_list = df['city'].astype('str')

text = 'Township'
head, sep, tail = text.partition('Township')


city_list.shape
pd.get_option("display.max_rows", None)
df

<ipython-input-11-078a2c8f9d73>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['city'] = df['city'].str.replace('\d+', '').str.strip()


,Unnamed: 0,city,population,violent_crime,murder,rape,robbery,assault,property_crime,burglary,larceny,gta,arson
0,0,Aberdeen Township,18700,15,0,3,5,7,164,7,145,12,0
1,1,Absecon,8436,8,0,1,2,5,236,24,207,5,0
2,2,Allendale,6795,0,0,0,0,0,38,8,26,4,0
3,3,Allenhurst,484,0,0,0,0,0,11,1,9,1,0
4,4,Allentown,1784,0,0,0,0,0,8,2,6,0,0
5,5,Alpha,2262,1,0,0,0,1,37,7,30,0,0
6,6,Alpine,1856,1,0,0,0,1,15,4,2,9,0
7,7,Andover Township,5870,0,0,0,0,0,17,3,13,1,0
8,8,Asbury Park,15437,179,2,10,47,120,525,59,457,9,0
9,9,Atlantic City,37593,323,11,29,182,101,1738,130,1519,89,2


In [7]:
def get_coordinates(city_list):
    """Takes a list of cities and returns a dictionary of the cities and their corresponding coordinates."""
    geolocator = Nominatim(user_agent='location script')
    dicto = {}
    
    for city in city_list:
        try:
            location = geolocator.geocode(city)
        except:
            raise Exception("There was a problem with the getCoordinates function")
        coordinate_values = (location.longitude, location.latitude)  #in geopandas, the x value corresponds to the longitude while the y value, the latitude(Just in case you were wondering why it was *location.longitude, location.latitude* and not the other way round )
        dicto[city] = coordinate_values #adding the coordinate pair to the dictionary at the end of every loop
    return dicto #finally retruns the dict 

In [8]:
#getting coordinates for each city in the list
city_coords_dict = get_coordinates(city_list)
city_coords_dict

AttributeError: 'NoneType' object has no attribute 'longitude'

In [ ]:
len(city_coords_dict)

In [ ]:
df.head()

In [ ]:
# crime rate per 100,000 is made as follows:
#(crime volume / population) * 100,000


#### 1. Add a total crime column

In [ ]:
df['total_crime'] = df.sum(axis=1)

In [ ]:
df.head()

#### 2. Making a new data frame that has only the neccesary columns for further operations

In [ ]:
new_df = df.filter(['city','total_crime','population'], axis=1)
new_df

### 3.  Add a column of the crime rate severity via (total crime / population) 

In [ ]:
new_df['crime_index'] = (new_df['total_crime'] / new_df['population'])

In [ ]:
new_df.round(5)

#### Next step is to create a final dataframe with following columns:
#### LATITUDE, LONGTITUDE, CRIME_INDEX

In [ ]:
#lets make the coordinates dictionary into dataframe and orient it for
#latitudes and longitudes
coords_df = pd.DataFrame.from_dict(city_coords_dict, orient='index')
coords_df

In [ ]:
coords_df.rename(columns={0: 'latitude', 1: 'longitude'}, inplace=True)

In [ ]:
coords_df

### join original dataframe row of 'crime_index' to new df of 'coordinates'

In [ ]:
coords_df['crime_index'] = new_df['crime_index'].values

In [ ]:
coords_df

In [ ]:
coords_df.insert(0, 'state', 'nj')
coords_df

In [ ]:
path = '/Users/pauliuskomskis/Desktop/github/greenzone/state-by-state-coords/'
coords_df.to_csv(path+'new_jersey-final.csv')